In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from glob import glob as glob
import os

In [ ]:
L = 5
num_samp = 10
im_size = np.array((192, 168))

In [ ]:
im_dirs = np.array(sorted(glob("CroppedYale/*")))
total_num_images = len(im_dirs)
im_idxs = np.random.choice(total_num_images, size=L, replace=False)

In [ ]:
def filter_files(d, num_samp, train=True):
    fnames = [os.path.basename(i)[:-4] for i in glob(d+"/*_P00A*.pgm")]
    acc_files = []
    for f in fnames:
        az = int(f[12:16])
        elev = int(f[17:20])
        cond = abs(az) <= 45 and abs(elev) <= 45
        if cond:
            acc_files.append(f)
    return np.random.choice(np.array(sorted(acc_files)), num_samp, replace=False)

In [ ]:
def fread(f):
    return plt.imread(f).flatten().T

In [ ]:
A = np.zeros((np.prod(im_size), L*num_samp))
train_fnames = []
# print(A.shape)
for i, d in enumerate(im_dirs[im_idxs]):
    for j, f in enumerate(filter_files(d, num_samp)):
        train_fnames.append(f)
        A[:,i*num_samp+j] = fread(d+"/"+f+".pgm")

In [ ]:
train_fnames

In [ ]:
def down_samp(A, ds_factor=8):
    im_size_down = np.ceil(im_size/ds_factor).astype(int)
    A_down = np.zeros((np.prod(im_size_down), A.shape[-1]))
    print(A_down.shape)
    for i in range(A.shape[-1]):
        A_down[:,i] = A[:,i].reshape(im_size)[::ds_factor, ::ds_factor].flatten()
    return A_down, im_size_down

In [ ]:
def down_samp_pca(A, dim=120):
#     im_size_down = np.ceil(im_size/ds_factor).astype(int)
#     A_down = np.zeros(dim, A.shape[-1])
#     print(A_down.shape)
    U, S, Vh = np.linalg.svd(A, full_matrices=False)
    print(U.shape, S.shape, Vh.shape)
#     U[:,:dim].T@A
#     for i in range(A.shape[-1]):
#         A_down[:,i] = A[:,i].reshape(im_size)[::ds_factor, ::ds_factor].flatten()
    return None

In [ ]:
A.shape

In [ ]:
down_samp_pca(A)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10, svd_solver="auto")
A_down = pca.fit_transform(A.T).T

In [ ]:
np.ceil(im_size/8).astype(int)

In [ ]:
A_down, im_size_down = down_samp(A, 16)

In [ ]:
plt.imshow(A_down[:,0].reshape(im_size_down))

In [ ]:
def delta(x, i):
    assert i < L
    out = np.zeros(len(x))
    idxs = slice(i*L, i*L+num_samp)
    out[idxs] = x[idxs]
    return out

In [ ]:
delta(np.ones(25), 4)

In [ ]:
y = A_down[:,31]

In [ ]:
y.shape

In [ ]:
def identity(A, y):
    A = A/np.linalg.norm(A, axis=0)
    prob = Lasso(fit_intercept=False)
    prob.fit(A, y)
    x_hat = prob.coef_
    r = np.zeros(L)
    for i in range(L):
        r[i] = np.linalg.norm(y-A@delta(x_hat, i))
#     print(r)
    return np.argmin(r)

In [ ]:
identity(A_down,y)

In [ ]:
test_fnames = []
for fname in im_dirs[im_idxs]:
    a = filter_files(fname, 1)
    while a in train_fnames:
        a = filter_files(fname, 1)
    test_fnames.append(fname+"/"+a[0]+".pgm")

In [ ]:
test_fnames

In [ ]:
identity(A_down, down_samp(fread(test_fnames[4])[:,None],16)[0])

In [ ]:
fread(test_fnames[4]).shape

In [ ]:
A_down.shape

In [ ]:
identity(A_down, pca.transform(fread(test_fnames[4])[:,None].T).T)

In [ ]:
# fread(test_fnames[0]).shape

In [ ]:
def robust_identity(A, y):
    A = A/np.linalg.norm(A, axis=0)
    m,n = A.shape
    B = np.hstack((A, np.eye(m)))
    print(B.shape)
    prob = Lasso(fit_intercept=False)
    prob.fit(B, y)
    w_hat = prob.coef_
    x_hat = w_hat[:n]
    e_hat = w_hat[n:]
    r = np.zeros(L)
    for i in range(L):
        r[i] = np.linalg.norm(y-e_hat-A@delta(x_hat, i))
    print(r-np.min(r))
    return np.argmin(r)

In [ ]:
y_test_noise = fread(test_fnames[2])[:,None]+np.random.normal(scale=500, size=(32256,1))

In [ ]:
plt.imshow(y_test_noise.reshape(im_size))

In [ ]:
robust_identity(A_down, down_samp(y_test_noise,16)[0])